In [ ]:
###########################################  GRUPO 1 ############################################

#####################################  Miembros del grupo  ######################################

# 20163197, Enrique Alfonso Pazos 
# 20191894, Ilenia Ttito
# 20151595, Rodrigo Ramos
# 20193469, Luis Eguzquiza 
# 20163377, Jean Niño de Guzmán

In [1]:
# Pregunta 1
import os
import pandas as pd
import numpy as np
import weightedcalcs as wc
from tqdm import tqdm 

# Directorio
user = os.getlogin()
os.chdir (f"C:/Users/{user}/Documents/GitHub/1ECO35_2022_2/Lab7")

# Leer un dta. de Stata:

'''2019'''
# Modulo 1
enaho_2019_modulo_1 = pd.read_stata(r"../../../Enaho/2019/687-Modulo01/687-Modulo01/enaho01-2019-100.dta", convert_categoricals=False)

# Modulo 34: sumaria
enaho_2019_modulo_34 = pd.read_stata(r"../../../Enaho/2019/687-Modulo34/687-Modulo34/sumaria-2019.dta", convert_categoricals=False)

'''2020'''
# Modulo 1
enaho_2020_modulo_1 = pd.read_stata(r"../../../Enaho/2020/737-Modulo01/737-Modulo01/enaho01-2020-100.dta", convert_categoricals=False)

# Modulo 34
enaho_2020_modulo_34 = pd.read_stata(r"../../../Enaho/2020/737-Modulo34/737-Modulo34/sumaria-2020.dta", convert_categoricals=False)

# Solo me quedo con las variables relevantes:
# Para el módulo 1: Año, Conglome, Vivienda
enaho_2019_modulo_1 = enaho_2019_modulo_1[["aÑo", "conglome", "vivienda", "hogar", "ubigeo"]]
enaho_2020_modulo_1 = enaho_2020_modulo_1[["aÑo", "conglome", "vivienda", "hogar", "ubigeo"]]
# Cambio el nombre de la columna año por year: 
enaho_2019_modulo_1 = enaho_2019_modulo_1.rename(columns={'aÑo': 'year'})
enaho_2020_modulo_1 = enaho_2020_modulo_1.rename(columns={'aÑo': 'year'})

# Para el módulo 34: conglome, vivienda, hogar, inghog1d, gashog2d, ld, mieperho
enaho_2019_modulo_34 = enaho_2019_modulo_34[["conglome", "vivienda", "hogar", "inghog1d", "gashog2d", "ld", "mieperho"]]
enaho_2020_modulo_34 = enaho_2020_modulo_34[["conglome", "vivienda", "hogar", "inghog1d", "gashog2d", "ld", "mieperho"]]

# El módulo 1 tiene características del hogar y será master data.
# El método de merge que uso es el keepus master, para el cual pongo how="left".
# De esa manera, me quedaré con las observaciones del módulo 1 que hagan y no match con el modulo 34.
# Las variables que usaré como criterio con Conglomerado, Vivienda y Hogar. 
# Emplearé como validate 1:1 porque ambas son a nivel hogar, de modo que se asume que el identificador no se repite.
# No es necesario poner el sufijo (suffix) al final.

merge_2019_1_34 = pd.merge(enaho_2019_modulo_1, enaho_2019_modulo_34, on = ["conglome", "vivienda", "hogar"], how = "left", validate = "1:1")
merge_2020_1_34 = pd.merge(enaho_2020_modulo_1, enaho_2020_modulo_34, on = ["conglome", "vivienda", "hogar"], how = "left", validate = "1:1")

# Hago el append entre las dos bases anteriores, de manera que la base del 2020 irá debajo del 2019.
append_2019_2020= merge_2019_1_34.append(merge_2020_1_34, ignore_index= True)

# Debo crear la variable corresponiente a departamento (la llamaré "dep").
# Para ello, debo extraer solo los dos primeros dígitos de ubigeo:
append_2019_2020["dep"] = append_2019_2020["ubigeo"].str[:2]
 
# Ahora, convierto en float todas las variables:
columns = append_2019_2020.columns  #Creo una lista con el nombre de las columnas 
# Iteraré con esta dentro de la base de datos para convertir cada variable en float
for i in columns:
    append_2019_2020[i]=append_2019_2020[i].astype(float)

# Extraigo los deflactores: 
deflactor_temporal = pd.read_stata(r"../../../Enaho/2020/737-Modulo34/737-Modulo34/ConstVarGasto-Metodologia actualizada/Gasto2020/Bases/deflactores_base2020_new.dta", convert_categoricals=False)

#Convierto todas las variables en float:
columns2 = deflactor_temporal.columns   #Creo una lista con el nombre de las columnas 
# Iteraré con esta dentro de la base de datos para convertir cada variable en float
for i in columns2:
    deflactor_temporal[i]=deflactor_temporal[i].astype(float)

#Cambiamos los nombres de las variables críticas para que lleve el mismo que la base de datos append_2019_2020:
deflactor_temporal = deflactor_temporal.rename(columns={'dpto': 'dep'})
deflactor_temporal = deflactor_temporal.rename(columns={'aniorec': 'year'})

# Usaré el método "left" porque en la master data está append_2019_2020.
# Usamos m:1 porque en la master data contiene multiples observaciones sobre los mismos departamentos por cada año.
# No usamos sufijos porque no tienen más variables en común además de las que pongo como criterios.

merge_final = pd.merge(append_2019_2020, deflactor_temporal, on = ['year', 'dep'], how = "left", validate = "m:1")

# Deflacto entre la variable "mieperho", 12, "ld", "i00"
merge_final["inghog1d"] = merge_final["inghog1d"].div(merge_final["mieperho"])
merge_final["gashog2d"] = merge_final["gashog2d"].div(merge_final["mieperho"])

merge_final["inghog1d"] = merge_final["inghog1d"].div(12)
merge_final["gashog2d"] = merge_final["gashog2d"].div(12)

merge_final["inghog1d"] = merge_final["inghog1d"].div(merge_final["ld"])
merge_final["gashog2d"] = merge_final["gashog2d"].div(merge_final["ld"])

merge_final["inghog1d"] = merge_final["inghog1d"].div(merge_final["i00"])
merge_final["gashog2d"] = merge_final["gashog2d"].div(merge_final["i00"])

C:\Users\Alfonso\AppData\Local\Temp\ipykernel_4588\3789377014.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  append_2019_2020= merge_2019_1_34.append(merge_2020_1_34, ignore_index= True)


In [2]:
"""
RESULTADO:
"""
merge_final

,year,conglome,vivienda,hogar,ubigeo,inghog1d,gashog2d,ld,mieperho,dep,i00,i01,i02,i03,i04,i05,i06,i07,i08
0,2019.0,5001.0,7.0,11.0,10101.0,942.402727,818.567861,0.815806,4.0,1.0,0.982802,0.961824,1.007068,0.988996,0.993981,0.996161,1.03648,1.011901,0.995908
1,2019.0,5001.0,17.0,11.0,10101.0,512.260548,421.077798,0.815806,6.0,1.0,0.982802,0.961824,1.007068,0.988996,0.993981,0.996161,1.03648,1.011901,0.995908
2,2019.0,5001.0,28.0,11.0,10101.0,NaN,NaN,NaN,NaN,1.0,0.982802,0.961824,1.007068,0.988996,0.993981,0.996161,1.03648,1.011901,0.995908
3,2019.0,5001.0,40.0,11.0,10101.0,802.508596,508.609816,0.815806,4.0,1.0,0.982802,0.961824,1.007068,0.988996,0.993981,0.996161,1.03648,1.011901,0.995908
4,2019.0,5001.0,50.0,11.0,10101.0,1473.884077,1121.111288,0.815806,4.0,1.0,0.982802,0.961824,1.007068,0.988996,0.993981,0.996161,1.03648,1.011901,0.995908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97286,2020.0,20305.0,52.0,11.0,250301.0,1031.372775,963.641790,0.924496,1.0,25.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
97287,2020.0,20305.0,73.0,11.0,250301.0,1158.018357,1043.523874,0.924496,1.0,25.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
97288,2020.0,20305.0,94.0,11.0,250301.0,NaN,NaN,NaN,NaN,25.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
97289,2020.0,20305.0,116.0,11.0,250301.0,473.441602,535.855695,0.924496,4.0,25.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [3]:
# Pregunta 2
import os # Para el usernanme y setear el direcotrio
import pandas as pd # Para utilizar la librería de pandas 
import numpy as np # Para utilizar la librería de numpy

In [4]:
user = os.getlogin()   # Username
os.chdir(f"C:/Users/{user}/Documents/Mis códigos") # Set directorio
# Leemos la base de datos de Stata usando pandas
df = pd.read_stata(r"../../../LENOVO/Documents/Mis códigos/enaho01a-2020-500.dta",
                           convert_categoricals = False)
# Debemos colocar convert_categoricals=False para que respete los value's label
# Utilizamos fillna para reemplazar todos los missing values con 0 
# Solo lo hacemos en las variables que serán útiles para el cálculo del salario por hora 
df[['i524e1','i538e1','i513t','i518']] = df[['i524e1','i538e1','i513t','i518']].fillna(0)
# Corroboramos nuestros resultados mostrando las primeras 5 filas de nuestras variables de interés
df[['i524e1','i538e1','i513t','i518']].head()

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'C:/Users/Alfonso/Documents/Mis códigos'

In [ ]:
# Creamos una nueva columna llamada salario_hora
# Rellenamos la nueva columna con la fórmula de salario por hora especificado en la tarea
df['salario_hora'] = (df['i524e1'] + df['i538e1']) / ((df['i513t'] + df['i518'])*52)
# Corroboramos nuestros resultados mostrando las 10 primeras filas de nuestras variables de interés 
df[['salario_hora','i524e1','i538e1','i513t','i518']].head(10)
# Observamos que en salario por hora hay varios 0, eso se tendrá que arreglar

In [ ]:
# Reemplazamos los 0 por Nan en la columna salario_hora
# Esto con la finalidad de indicar que no hay datos para esas filas 
df.loc[df['salario_hora'] == 0, 'salario_hora'] = np.nan
# Corroboramos nuestros resultados mostrando las 10 primeras filas 
df[['salario_hora','i524e1','i538e1','i513t','i518']].head(10)

In [ ]:
# Observamos que en total hay 31 226 datos faltantes en la columna salario_hora
# Sin embargo, eso no es problema, pues aún tenemos como 60 000 observaciones disponibles 
df['salario_hora'].isnull().sum()